In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time
from scipy.stats import rankdata
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.datasets import make_sparse_coded_signal
from scipy.stats import bernoulli

In [2]:
def GenerateData(N, M):
    a = np.zeros(M)
    b = np.zeros(M)
    m = np.zeros(M)
    n = np.zeros(M)
    
    for i in range(M-10):
        m[i] = np.random.binomial(N, 1/3)
        n[i] = np.random.binomial(N-m[i], 1/2)
        a[i] = np.random.binomial(m[i], 1/2)
        b[i] = np.random.binomial(n[i], 1/2)
    
    for i in range(M-10,M):
        m[i] = np.random.binomial(N, 1/3)
        n[i] = np.random.binomial(N-m[i], 1/2)
        a[i] = np.random.binomial(m[i], 3/4)
        b[i] = np.random.binomial(n[i], 3/4)
    
    stats = np.zeros(M)
    order = np.zeros(M)
    
    for i in range(M):
        order[i] = i
        if m[i] == 0:
            f = 0
        else:
            f = (2*a[i]-m[i])**2/m[i]
        if n[i] == 0:
            s = 0
        else:
            s = (2*b[i]-n[i])**2/n[i]
        if m[i] + n[i] == N:
            t = 0
        else:
            t = (2*a[i]+2*b[i]-m[i]-n[i])**2/(N-m[i]-n[i])
        
        stats[i] = f + s + t
    
    x = np.argsort(stats)
    #for i in range(10):
    #    print(x[M-1-i])
    
    stats = sorted(stats, reverse=True)
    #print(sorted_stats)
    
    return stats

In [3]:
def HaarMatrix(N):
    A = np.zeros((N,N))
    for j in range(0,N):
        A[0,j] = 1/np.sqrt(N)
    for i in range(1,N):
        for j in range(0,N):
            if N*(i-2**(np.floor(math.log2(i))))/(2**(np.floor(math.log2(i)))) <= j and j < N*(i-2**(np.floor(math.log2(i))) + 1/2)/(2**(np.floor(math.log2(i)))):
                A[i,j] = 2**(np.floor(math.log2(i))/2)/np.sqrt(N)
            elif N*(i-2**(np.floor(math.log2(i))) + 1/2)/(2**(np.floor(math.log2(i)))) <= j and j < N*(i-2**(np.floor(math.log2(i))) + 1)/(2**(np.floor(math.log2(i)))):
                A[i,j] = -2**(np.floor(math.log2(i))/2)/np.sqrt(N)
    return A

In [4]:
def BernoulliMatrix(K,N):
    phi = np.zeros((K,N))
    
    for i in range(K):
        for j in range(N):
            phi[i][j] = bernoulli.rvs(0.5, size=1)

    return (phi - 1/2) / (np.sqrt(K) / 2)

In [5]:
def OMP(A, y, N, K):
    x = np.zeros(N)
    S = np.zeros(N, dtype = np.uint8)
    r = y
    rr = np.dot(r,r)
    
    for i in range(K):
        err = rr - np.dot(A[:,S == 0].T, r) ** 2
        ndx = np.where(S == 0)[0]
        S[ndx[err.argmin()]] = 1
        
        As = A[:, S == 1]
        pinv = np.linalg.pinv(np.dot(As, As.T))
        x[S==1] = np.dot(As.T, np.dot(pinv, y))
        
        r = y - np.dot(A, x)
        rr = np.dot(r,r)
    
    return x

def OMPCL(A, y, N, K):
    x = np.zeros(N)
    S = np.zeros(N, dtype = np.uint8)
    
    for i in range(K):
        for j in range(K):
            S[j] = 1
        
        As = A[:, S == 1]
        pinv = np.linalg.pinv(np.dot(As, As.T))
        x[S==1] = np.dot(As.T, np.dot(pinv, y))
    
    return x

In [6]:
def determineS(n,d,psi,eta):
    xx = np.dot(psi.T, d)
    
    s = 0
    for i in range(n):
        if xx[i] > eta:
            s += 1
    
    return s

In [7]:
def CompressiveMechanism(g, deltag, K, epsilon,eta):
    m = np.size(g)
    n = int(2**(np.ceil(math.log2(m))))
    
    #print(n)
    
    d = np.zeros(n)
    for i in range(m):
        d[i] = g[i]
    
    psi = HaarMatrix(n)
    
    s = determineS(n,d,psi,eta)
    k = int(np.floor(s*math.log(n/s)))
    
    phi = BernoulliMatrix(k,n)
    A = np.dot(phi, psi)
    
    y = np.dot(phi, d)
    pri_y = y + np.random.laplace(loc = 0.0, scale = 2*K*deltag*np.sqrt(k)/epsilon, size=k)
    
    pri_x = OMP(A, pri_y, n, s)
    
    pri_d = np.dot(psi, pri_x)
    pri_g = np.zeros(m)
    for i in range(m):
        pri_g = pri_d
    
    return pri_g

In [8]:
def CompLaplace(g, deltag, s, K, epsilon):
    m = np.size(g)
    n = int(2**(np.ceil(math.log2(m))))
    k = int(np.floor(s*math.log(n/s)))
    
    t = g[int(s)-1]
    
    x = np.zeros(n)
    for i in range(n):
        if i < m and g[i] >= t:
            x[i] = g[i]
        else:
            x[i] = 0
    
    phi = BernoulliMatrix(k,n)
    psi = HaarMatrix(n)
    A = np.dot(phi, psi)
    
    y = np.dot(A, x)
    
    noise = (2*K/epsilon)*np.sqrt(k)*deltag*((1+(np.sqrt(n)-1)*(np.sqrt(2)+1))/np.sqrt(n))
    pri_y = y + np.random.laplace(loc = 0.0, scale = noise, size = k)
    
    pri_x = OMPCL(A, pri_y, n, s)
    
    pri_g = g + np.random.laplace(loc = 0.0, scale = 2*K*deltag/epsilon, size=m)
    
    for i in range(m):
        if g[i] >= t:
            pri_g[i] = pri_x[i]
    
    return pri_g

In [9]:
def lap_noise_normal(data, K, epsilon):
    pri_data = data + np.random.laplace(loc = 0.0, scale = 2*K*(4*N/(N+2))/epsilon, size=np.size(data))
    
    return pri_data

In [10]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_mec(stats, s, m, K, epsilon):
    h = np.zeros(m)
    for i in range(m):
        h[i] = stats[i]
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * h[i] / (2*K*s))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        h[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [11]:
K = 3
epsilon = 0.5
eta = 10
s = 10

N = 500
M = 10000

stats = GenerateData(N,M)

RT = np.zeros(4)

for i in range(5):
    #start = time.time()
    #pri_stats = CompressiveMechanism(stats, 4*N/(N+2), K, epsilon, eta)
    #end = time.time()
    #RT[0] += end-start
    
    #print("Comp. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = CompLaplace(stats, 4*N/(N+2), s, K, epsilon)
    end = time.time()
    RT[1] += end-start
    print("Comp. + Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = lap_noise_normal(stats, K, epsilon)
    end = time.time()
    RT[2] += end-start
    print("Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = exp_mec(stats, 4*N/(N+2), M, K, epsilon)
    end = time.time()
    RT[3] += end-start
    print("Exp. = ", end-start, "[sec]")
    print("------------")

print("------------")
print("Average")
RT /= 5
#print("Comp. = ", RT[0], "[sec]")
print("Comp. + Lap. = ", RT[1], "[sec]")
print("Lap. = ", RT[2], "[sec]")
print("Exp. = ", RT[3], "[sec]")

Comp. + Lap. =  2093.366886138916 [sec]
Lap. =  0.002080202102661133 [sec]
Exp. =  0.032414913177490234 [sec]
------------
Comp. + Lap. =  2082.3937289714813 [sec]
Lap. =  0.001825094223022461 [sec]
Exp. =  0.0315859317779541 [sec]
------------
Comp. + Lap. =  2037.0446028709412 [sec]
Lap. =  0.0019409656524658203 [sec]
Exp. =  0.03224611282348633 [sec]
------------
Comp. + Lap. =  2070.6203660964966 [sec]
Lap. =  0.0020079612731933594 [sec]
Exp. =  0.03139901161193848 [sec]
------------
Comp. + Lap. =  2032.5822458267212 [sec]
Lap. =  0.001834869384765625 [sec]
Exp. =  0.03167104721069336 [sec]
------------
------------
Average
Comp. + Lap. =  2063.2015659809113 [sec]
Lap. =  0.0019378185272216797 [sec]
Exp. =  0.0318634033203125 [sec]
